In [63]:
import pandas as pd
import numpy as np

In [95]:
#importazione del dataset originale
df = pd.read_csv("data.csv", dtype = {'store_and_fwd_flag': 'category'})
df['tpep_pickup_datetime']=pd.to_datetime(df['tpep_pickup_datetime'], format='mixed')
df['tpep_dropoff_datetime']=pd.to_datetime(df['tpep_dropoff_datetime'], format='mixed')
#rimozione righe null
df = df.dropna(subset=['VendorID','passenger_count','store_and_fwd_flag','payment_type'])
#creazione colonna per gli intervalli di tempo
df['pick_up_intervals'] = ( df['tpep_pickup_datetime'].dt.hour * 60 + df['tpep_pickup_datetime'].dt.minute ) // 30

In [9]:
#da qui iniziano le modifiche:
#si cercano relazioni tra 'fare_amount'<0 e 'tip_amount'

In [ ]:
#1/3) 'tip_amount'<0

In [ ]:
#quando (tip_amount < 0) => (tutte le colonne legate a prezzi sono <= 0)

#per queste righe può aver senso cambiare tutti i valori in positivi
#sono solo 170

In [97]:
len(df[ (df['fare_amount'] < 0) & (df['tip_amount'] < 0) ])

170

In [99]:
primo_caso = df[df['tip_amount']<0][['fare_amount','extra','mta_tax','tip_amount','tolls_amount','improvement_surcharge','total_amount','congestion_surcharge']]
primo_caso.head()

,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
17630,-3.5,-0.5,-0.5,-0.80,0.0,-0.3,-5.60,0.0
82398,-2.5,0.0,-0.5,-0.66,0.0,-0.3,-3.96,0.0
139505,-2.5,-1.0,-0.5,-0.86,0.0,-0.3,-5.16,0.0
139507,-2.5,-1.0,-0.5,-0.08,0.0,-0.3,-4.38,0.0
191332,-8.0,0.0,-0.5,-7.00,0.0,-0.3,-15.80,0.0


In [ ]:
#comando per cambiare di segno le colonne ['tip_amount', 'fare_amount'], quando 'tip_amount'<0

In [101]:
df.loc[df['tip_amount'] < 0, ['tip_amount', 'fare_amount']] *= -1

In [ ]:
#2/3) 'tip_amount'>0

In [103]:
#un'altra cosa problematica è questa: 
len( df[ (df['fare_amount'] < 0) & (df['tip_amount'] > 0) ] )

#cioè ci sono 208 osservazioni con:
#tip POSITIVE e tutte le altre colonne legate a spese NEGATIVE

#se prendiamo per vero che anche qui tutti i prezzi dovrebbero essere positivi e sono stati inseriti male per qualche motivo,
#si può semplicemente cambiare il segno alle colonne di interesse
#(bisognerebbe ricalcolare il total_amount ma sticazzi tanto non serve)

208

In [105]:
secondo_caso = df[ (df['fare_amount'] < 0) & (df['tip_amount'] > 0) ][['fare_amount','extra','mta_tax','tip_amount','tolls_amount','improvement_surcharge','total_amount','congestion_surcharge']]
secondo_caso.head()

,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
34639,-27.5,-0.5,-0.5,5.00,0.0,-0.3,-26.30,-2.5
53091,-70.0,0.0,0.0,14.06,0.0,-0.3,-56.24,0.0
53093,-70.0,0.0,0.0,14.06,0.0,-0.3,-56.24,0.0
128011,-62.5,0.0,0.0,15.70,0.0,-0.3,-47.10,0.0
171300,-20.5,-0.5,-0.5,4.86,0.0,-0.3,-19.44,-2.5


In [ ]:
#comando per cambiare di segno la colonna ['fare_amount'], quando 'tip_amount'>0 e 'fare_amount'<0

In [107]:
df.loc[(df['fare_amount'] < 0) & (df['tip_amount'] > 0), ['fare_amount']] *= -1

In [ ]:
#3/3) 'tip_amount'==0

In [109]:
#l'ultima casistica da indagare è quando fare_amount è < 0 e tip_amount è nullo:
len( df[ (df['fare_amount'] < 0) & (df['tip_amount'] == 0) ])
#comunque pochissimi rispetto al dataset

19063

In [115]:
#rientrano nella casistica 1: hanno quindi tutte le righe dei prezzi negative, 
#ad eccezione di 2 osservazioni che hanno tutto negativo ma 'extra' > 0, però sono solo due...

#infatti, la lunghezza delle righe che hanno 'fare_amount'<0, 'tip_amount'== 0 e tutte le colonne di prezzo <= 0 è molto simile a quel 19063
len(df[ (df['fare_amount'] < 0) & 
        (df['extra'] <= 0) & (df['mta_tax'] <= 0) & 
        (df['tolls_amount'] <= 0) & (df['improvement_surcharge'] <= 0) & 
        (df['total_amount'] <= 0) & (df['congestion_surcharge'] <= 0) &
        (df['tip_amount'] == 0) ])

19061

In [ ]:
#si può anche qui cambiare il segno di fare_amount

In [117]:
df.loc[(df['fare_amount'] < 0) & (df['tip_amount'] == 0), ['fare_amount']] *= -1

In [ ]:
#I risultati di media sono ovviamente molto diversi da quelli ottenuti con i valori negativi:

In [121]:
#Testo la Soluzione
avg_df = df.groupby(['payment_type','pick_up_intervals'])['fare_amount'].mean()
avg_df

payment_type  pick_up_intervals
1.0           0                    13.869142
              1                    13.472337
              2                    12.824603
              3                    12.357974
              4                    12.008589
                                     ...    
4.0           44                   12.017559
              45                   14.269400
              46                   13.013658
              47                   15.210162
5.0           35                    0.000000
Name: fare_amount, Length: 193, dtype: float64